In [2]:
import os;
import json;
import builtins
import duckdb
import pandas as pd;
import matplotlib.pyplot as plt

def get_dataset_from_testcase(run):
    pos = run.find("_threads")
    return run[0:pos]

dirs = [
    '/home/chesetti/Repos/LearnedJoinDiskExp/sponge/index_stats',
    '/home/chesetti/Repos/LearnedJoinDiskExp/sponge_alex/testrun'
]
runs = []
for dir in dirs:
 for test_case in os.listdir(dir):
    if test_case == 'build':
        continue
    for run_name in os.listdir(os.path.join(dir, test_case, 'outputs', 'results')):
        if (run_name=="run"):
           continue
        json_file = builtins.open(os.path.join(dir, test_case, 'outputs', 'results',  run_name))
        run = (json.load(json_file))
        run['dataset'] = get_dataset_from_testcase(test_case)
        runs.append(run)
    for run_name in os.listdir(os.path.join(dir, test_case, 'outputs', 'results', 'run')):
        json_file = builtins.open(os.path.join(dir, test_case, 'outputs', 'results', 'run', run_name))
        run = (json.load(json_file))
        run['dataset'] = get_dataset_from_testcase(test_case)
        runs.append(run)
df = pd.json_normalize(runs)
print(df.columns)
display(df)

Index(['command', 'dataset', 'result.checksum', 'result.duration_ns',
       'result.duration_sec', 'result.inner_disk_fetch',
       'result.inner_disk_fetch_size', 'result.inner_total_bytes_fetched',
       'result.num_output_keys', 'result.outer_disk_fetch',
       'result.outer_disk_fetch_size', 'result.outer_total_bytes_fetched',
       'spec.algo', 'spec.algo_name', 'spec.check_checksum', 'spec.common_key',
       'spec.index.epsilon', 'spec.index.search', 'spec.index.type',
       'spec.inner_table', 'spec.key_size', 'spec.key_type',
       'spec.load_sstable_in_mem', 'spec.name', 'spec.num_threads',
       'spec.outer_table', 'spec.result_path', 'spec.value_size',
       'spec.write_result_to_disk', 'iostat.bytes_read', 'iostat.bytes_wrtn',
       'spec.index.dataset', 'spec.index.leaf_size_in_pages'],
      dtype='object')


,command,dataset,result.checksum,result.duration_ns,result.duration_sec,result.inner_disk_fetch,result.inner_disk_fetch_size,result.inner_total_bytes_fetched,result.num_output_keys,result.outer_disk_fetch,...,spec.name,spec.num_threads,spec.outer_table,spec.result_path,spec.value_size,spec.write_result_to_disk,iostat.bytes_read,iostat.bytes_wrtn,spec.index.dataset,spec.index.leaf_size_in_pages
0,sponge/index_stats/build/benchmark_runner spon...,books,0,250523895726,250.523896,4217887,8192,34552929456,800000000,3125000,...,inlj_flatpgm2561run_0,1,sponge/index_stats/books_threads=1/inputs/input1,sponge/index_stats/books_threads=1/outputs/inl...,8,True,0,0,NaN,NaN
1,sponge/index_stats/build/benchmark_runner spon...,books,0,276726897562,276.726898,23514294,8192,192629092352,800000000,3125000,...,inlj_sampledflatpgm2561run_0,1,sponge/index_stats/books_threads=1/inputs/input1,sponge/index_stats/books_threads=1/outputs/inl...,8,True,0,0,NaN,NaN
2,sponge/index_stats/build/benchmark_runner spon...,books,0,242097925331,242.097925,7237731,4096,29645542928,800000000,3125000,...,inlj_rmi1run_0,1,sponge/index_stats/books_threads=1/inputs/input1,sponge/index_stats/books_threads=1/outputs/inl...,8,True,0,0,books_800M_uint64,NaN
3,sponge/index_stats/build/benchmark_runner spon...,books,0,302415678665,302.415679,3125000,4096,12800000000,800000000,3125000,...,inlj_btree2561run_0,1,sponge/index_stats/books_threads=1/inputs/input1,sponge/index_stats/books_threads=1/outputs/inl...,8,True,8000,0,NaN,1.0
4,sponge/index_stats/build/benchmark_runner spon...,books,0,844379883186,844.379883,818423078,4096,3352260871792,800000000,3125000,...,inlj_radixspline2561run_0,1,sponge/index_stats/books_threads=1/inputs/input1,sponge/index_stats/books_threads=1/outputs/inl...,8,True,0,0,NaN,NaN
5,sponge/index_stats/build/benchmark_runner spon...,lognormal,0,44928460686,44.928461,1218075,8192,9978459696,200000000,781250,...,inlj_flatpgm2561run_0,1,sponge/index_stats/lognormal_threads=1/inputs/...,sponge/index_stats/lognormal_threads=1/outputs...,8,True,0,0,NaN,NaN
6,sponge/index_stats/build/benchmark_runner spon...,lognormal,0,50644293816,50.644294,2637863,8192,21609371648,200000000,781250,...,inlj_sampledflatpgm2561run_0,1,sponge/index_stats/lognormal_threads=1/inputs/...,sponge/index_stats/lognormal_threads=1/outputs...,8,True,0,0,NaN,NaN
7,sponge/index_stats/build/benchmark_runner spon...,lognormal,0,1164170965606,1164.170966,1230234292,4096,5039039410432,200000000,781250,...,inlj_rmi1run_0,1,sponge/index_stats/lognormal_threads=1/inputs/...,sponge/index_stats/lognormal_threads=1/outputs...,8,True,0,0,lognormal_200M_uint64,NaN
8,sponge/index_stats/build/benchmark_runner spon...,lognormal,0,60059725177,60.059725,781250,4096,3200000000,200000000,781250,...,inlj_btree2561run_0,1,sponge/index_stats/lognormal_threads=1/inputs/...,sponge/index_stats/lognormal_threads=1/outputs...,8,True,0,0,NaN,1.0
9,sponge/index_stats/build/benchmark_runner spon...,lognormal,0,414802703797,414.802704,403460345,4096,1652573358224,200000000,781250,...,inlj_radixspline2561run_0,1,sponge/index_stats/lognormal_threads=1/inputs/...,sponge/index_stats/lognormal_threads=1/outputs...,8,True,0,0,NaN,NaN


In [3]:

epsilon = {
    "pgm256": 256,
    "pgm1024": 1024,
    "pgm2048": 2048,
    "flatpgm256": 256,
    "flatpgm1024": 1024,
    "flatpgm2048": 2048,
    "flatpgm4096": 4096,
    "flatpgm8192": 8192,
    "hashjoin": 0,
    "btree": 0,
    "sj": 0,
    "sj2": 0,
    "standard_merge": 0
};
def indexType(x):
    if "pgm" in x:
        return "PGM"
    elif "btree" in x:
        return "BTREE"
    elif "alex" in x:
        return "ALEX"
    elif "RMI" in x:
        return "RMI"
    return "NA"

df["threads"] = df["spec.num_threads"]
df["duration_sec"] = df["result.duration_ns"] / (1000000000)
df["inner_disk_fetch"] = df["result.inner_disk_fetch"]
df['result_num_keys'] = df['result.num_output_keys']
df["ratio"] = df["spec.common_key"]
df["algo"] = df["spec.algo_name"]
df["epsilon"] = df["spec.index.epsilon"]
df["inner_bytes_fetched"] = df["result.inner_total_bytes_fetched"]
df["index_type"] = df["spec.algo_name"].map(lambda x: indexType(x))
df['disk_read_gb'] = df['iostat.bytes_read']/1e9
df['disk_wrtn_gb'] = df['iostat.bytes_wrtn']/1e9
#df['cdisk_read_gb'] = df['cgroup_iostat.bytes_read']/1e9
#df['cdisk_wrtn_gb'] = df['cgroup_iostat.bytes_wrtn']/1e9

df['dataset'].unique()
df['result.num_output_keys'].unique()
duckdb.sql("SELECT * FROM df WHERE dataset='fb'")



┌──────────────────────┬─────────┬─────────────────┬────────────────────┬───┬────────────┬──────────────┬──────────────┐
│       command        │ dataset │ result.checksum │ result.duration_ns │ … │ index_type │ disk_read_gb │ disk_wrtn_gb │
│       varchar        │ varchar │      int64      │       int64        │   │  varchar   │    double    │    double    │
├──────────────────────┼─────────┼─────────────────┼────────────────────┼───┼────────────┼──────────────┼──────────────┤
│ sponge/index_stats…  │ fb      │               0 │        48348033985 │ … │ PGM        │          0.0 │          0.0 │
│ sponge/index_stats…  │ fb      │               0 │        63662366724 │ … │ PGM        │          0.0 │          0.0 │
│ sponge/index_stats…  │ fb      │               0 │        43100987233 │ … │ PGM        │          0.0 │          0.0 │
│ sponge/index_stats…  │ fb      │               0 │        46708647285 │ … │ BTREE      │          0.0 │          0.0 │
│ sponge/index_stats…  │ fb     

In [4]:
tmpdf = duckdb.sql("SELECT DATASET, RATIO,ALGO,AVG(DURATION_SEC) FROM df GROUP BY ratio,algo,dataset ORDER BY ratio,algo").df()
display(tmpdf)

def print_plot(indexes, datasets, stat):
    print(stat)
    tmp = duckdb.sql(f"SELECT {stat}, algo, dataset FROM df").df();
    for index in indexes:
        plot_line = f'\\addplot[{index[1]}] coordinates '
        plot_line += '{'
        for dataset in datasets:
            tmp = duckdb.sql(f"SELECT AVG({stat}) as v FROM df WHERE algo='{index[0]}' AND dataset='{dataset['col']}'").df();
            if len(tmp) == 1:
                value = float(tmp.loc[0]['v'])
            else:
                display(tmp)
                value = 0
            plot_line += f"({dataset['name']}, {value}) "
            
        plot_line += '};'
        print(plot_line)
    print()

print_plot(
    [('inlj_btree256', 'btree256'), 
     ('inlj_flatpgm256', 'pgm256'), ('inlj_sampledflatpgm256', 'sampledflatpgm256'), ('inlj_radixspline256', 'radixspline256'), ('alex_inlj', 'alex'), ('inlj_rmi', 'rmi')], 
    [
        {'col': 'uniform_dense', 'name': 'udense'},
        {'col': 'uniform_sparse', 'name': 'usparse'},
        {'col': 'normal', 'name': 'lognormal'},
        {'col': 'lognormal', 'name': 'normal'},
    ], 
    'duration_sec')

print_plot(
    [('inlj_btree256', 'btree256'), 
     ('inlj_flatpgm256', 'pgm256'), ('inlj_sampledflatpgm256', 'sampledflatpgm256'), ('inlj_radixspline256', 'radixspline256'), ('alex_inlj', 'alex'), ('inlj_rmi', 'rmi')], 
    [
        {'col': 'fb', 'name': 'FB'},
        {'col': 'wiki', 'name': 'Wiki'},
        {'col': 'osm', 'name': 'OSM'},
        {'col': 'books', 'name': 'Books'},
    ], 
    'duration_sec')


,dataset,ratio,algo,avg(DURATION_SEC)
0,wiki,1,alex_inlj,242.273546
1,uniform_sparse,1,alex_inlj,561.878275
2,lognormal,1,alex_inlj,558.106249
3,uniform_dense,1,alex_inlj,557.559938
4,normal,1,alex_inlj,550.907869
5,fb,1,alex_inlj,601.456905
6,books,1,inlj_btree256,302.415679
7,wiki,1,inlj_btree256,22.590842
8,osm,1,inlj_btree256,312.474651
9,uniform_sparse,1,inlj_btree256,53.078518


duration_sec
\addplot[btree256] coordinates {(udense, 60.157650793) (usparse, 53.078517727) (lognormal, 60.773513641) (normal, 60.059725177) };
\addplot[pgm256] coordinates {(udense, 37.422427809) (usparse, 42.79050186) (lognormal, 41.958182799) (normal, 44.928460686) };
\addplot[sampledflatpgm256] coordinates {(udense, 44.548015201) (usparse, 44.725962037) (lognormal, 43.906248487) (normal, 50.644293816) };
\addplot[radixspline256] coordinates {(udense, 409.019008898) (usparse, 238.933253038) (lognormal, 229.786962303) (normal, 414.802703797) };
\addplot[alex] coordinates {(udense, 557.559938248) (usparse, 561.878274834) (lognormal, 550.907869286) (normal, 558.106249159) };
\addplot[rmi] coordinates {(udense, 102.321590804) (usparse, 35.905478947) (lognormal, 37.378590809) (normal, 1164.170965606) };

duration_sec
\addplot[btree256] coordinates {(FB, 53.64762255066666) (Wiki, 22.590841694) (OSM, 312.474651447) (Books, 302.415678665) };
\addplot[pgm256] coordinates {(FB, 45.188915041) 